In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

In [10]:
data_dir = Path("../data/photographed_data/")
render_dir = Path("../data/mediapipe_render/")
vis_dir = Path("../data/mediapipe_visualize/")
# log_dir = Path("../logs/evaluations/runs/image-translate_aug2/best")
log_dir = Path("../logs/evaluations/runs/render-translate_aug2/best")
save_dir = log_dir.joinpath("incorrect_img")
save_dir.mkdir(exist_ok=True)
df = pd.read_csv(log_dir.joinpath("result.csv"))
df.head()

,PATH,TARGETS,PREDs
0,G/WIN_20221122_16_14_30_Pro_00000000.jpg,0,0
1,G/WIN_20221122_16_14_30_Pro_00000001.jpg,0,1
2,G/WIN_20221122_16_14_30_Pro_00000002.jpg,0,0
3,G/WIN_20221122_16_14_30_Pro_00000003.jpg,0,1
4,G/WIN_20221122_16_14_30_Pro_00000004.jpg,0,1


In [3]:
targets = df["TARGETS"].values
preds = df["PREDs"].values
correct_path = df[targets == preds]["PATH"]
incorrect_df = df[targets != preds]
incorrect_df

,PATH,TARGETS,PREDs
1,G/WIN_20221122_16_14_30_Pro_00000001.jpg,0,1
3,G/WIN_20221122_16_14_30_Pro_00000003.jpg,0,1
4,G/WIN_20221122_16_14_30_Pro_00000004.jpg,0,1
5,G/WIN_20221122_16_14_30_Pro_00000005.jpg,0,1
6,G/WIN_20221122_16_14_30_Pro_00000006.jpg,0,1
...,...,...,...
1776,H/WIN_20221122_15_59_43_Pro_00001187.jpg,1,0
1777,H/WIN_20221122_15_59_43_Pro_00001188.jpg,1,0
1780,H/WIN_20221122_15_59_43_Pro_00001191.jpg,1,0
1781,H/WIN_20221122_15_59_43_Pro_00001192.jpg,1,0


In [12]:
category = ["not_contact", "contact"]
is_render = False
full = True
for i, relative_img_path in enumerate(tqdm(incorrect_df["PATH"].values)):
    target, pred = incorrect_df.iloc[i,1:]
    if is_render:
        img_path = render_dir.joinpath(relative_img_path) 
    elif not full:
        img_path = data_dir.joinpath(relative_img_path) 
    else:
        img_path1 = render_dir.joinpath(relative_img_path) 
        img_path2 = vis_dir.joinpath(relative_img_path) 
    tag = category[target]
    image = np.array(Image.open(img_path))
    if not full:
        plt.imshow(image)
        plt.title(f"GT:{tag}, pred:{category[pred]}")
        # plt.show()
        plt.savefig(save_dir.joinpath(img_path.name))
    else:
        image1 = np.array(Image.open(img_path1))
        image2 = np.array(Image.open(img_path2))
        fig, ax = plt.subplots(2, 1, figsize=(4.8, 6.4))
        ax[0].imshow(image2)
        ax[1].imshow(image1)
        fig.suptitle(f"GT:{tag}, pred:{category[pred]}")
        plt.savefig(save_dir.joinpath(img_path1.name))
    plt.clf()
    plt.close()
    # break

100%|██████████| 299/299 [01:23<00:00,  3.60it/s]
